## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 4
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_4


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 16.2 s


Team  games_played  season  offense_points_per_game  \
0   Arizona Cardinals             4    2021                     35.0   
1  Kansas City Chiefs             4    2021                     33.5   

   offense_passing_AVG  offense_passing_YDS/G  offense_passing_RTG  \
0                  9.5                  304.0                115.0   
1                  8.6                  297.3                119.6   

   offense_receiving_AVG  offense_rushing_YDS/G  defense_points_per_game  ...  \
0                   12.5                  136.5                     21.3  ...   
1                   11.9                  130.3                     31.3  ...   

   defense_receiving_AVG  defense_rushing_YDS/G  defense_passing_SYL_per_game  \
0                   10.4                  135.8                         18.75   
1                   12.7                  146.0                         15.25   

   offense_downs_Third Downs_ATT_per_game  \
0                                   11.00   
1                                    9.75   

   defense_downs_Third Downs_ATT_per_game  \
0                                   11.50   
1                                   10.25   

   defense_downs_Fourth Downs_ATT_per_game  defense_pass_to_rush_ratio  \
0                                     1.25                    1.415842   
1                                     2.00                    1.296296   

   defense_downs_First Downs_penalty_ratio  offense_passing_TD_per_game  \
0                                 0.060241                         2.25   
1                                 0.066038                         3.50   

   offense_pass_TD_to_INT  
0                    2.25  
1                    3.50  

[2 rows x 21 columns]

In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

Team  season week_marker   pred_rf  pred_gbm  pred_lgb  \
0  Arizona Cardinals    2021      week_4  0.303700  0.310763  0.426321   
9   Los Angeles Rams    2021      week_4  0.322024  0.285940  0.426003   
5      Buffalo Bills    2021      week_4  0.287461  0.250836  0.298142   

   pred_catboost  pred_mean  
0       0.350658   0.347861  
9       0.348209   0.345544  
5       0.318303   0.288686

In [8]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [9]:
predictions

Team  season week_marker   pred_rf  pred_gbm  pred_lgb  \
0      Arizona Cardinals    2021      week_4  0.303700  0.310763  0.426321   
9       Los Angeles Rams    2021      week_4  0.322024  0.285940  0.426003   
5          Buffalo Bills    2021      week_4  0.287461  0.250836  0.298142   
1     Kansas City Chiefs    2021      week_4  0.288481  0.254102  0.304189   
21    Cincinnati Bengals    2021      week_4  0.224263  0.221437  0.303210   
2         Dallas Cowboys    2021      week_4  0.250650  0.218320  0.205664   
4      Las Vegas Raiders    2021      week_4  0.217670  0.149362  0.243658   
11      Cleveland Browns    2021      week_4  0.191935  0.172743  0.287764   
6   Tampa Bay Buccaneers    2021      week_4  0.239626  0.186266  0.190762   
12     Carolina Panthers    2021      week_4  0.203313  0.192942  0.239514   
16        Denver Broncos    2021      week_4  0.197615  0.120856  0.063754   
13       New York Giants    2021      week_4  0.175647  0.090976  0.145495   
7    Philadelphia Eagles    2021      week_4  0.112906  0.075567  0.189738   
3       Baltimore Ravens    2021      week_4  0.140614  0.076995  0.129535   
15   San Francisco 49ers    2021      week_4  0.111008  0.070918  0.103147   
10  Los Angeles Chargers    2021      week_4  0.123071  0.071930  0.086337   
14     Minnesota Vikings    2021      week_4  0.114349  0.069705  0.099102   
18      Seattle Seahawks    2021      week_4  0.116989  0.064654  0.090032   
19            Washington    2021      week_4  0.068949  0.052783  0.153425   
24     Green Bay Packers    2021      week_4  0.087885  0.052523  0.065748   
27    New Orleans Saints    2021      week_4  0.085057  0.045533  0.059271   
31         Chicago Bears    2021      week_4  0.061856  0.051503  0.065763   
28         New York Jets    2021      week_4  0.055933  0.050887  0.067349   
17         Detroit Lions    2021      week_4  0.042269  0.032428  0.041686   
30        Miami Dolphins    2021      week_4  0.030844  0.042150  0.037912   
8       Tennessee Titans    2021      week_4  0.009638  0.033842  0.045072   
25  New England Patriots    2021      week_4  0.014089  0.029463  0.039169   
23       Atlanta Falcons    2021      week_4  0.009345  0.028657  0.041113   
26   Pittsburgh Steelers    2021      week_4  0.007317  0.029613  0.037912   
22  Jacksonville Jaguars    2021      week_4  0.002892  0.026578  0.038077   
29        Houston Texans    2021      week_4  0.004172  0.027730  0.037912   
20    Indianapolis Colts    2021      week_4  0.001479  0.025585  0.037912   

    pred_catboost  pred_mean  
0        0.350658   0.347861  
9        0.348209   0.345544  
5        0.318303   0.288686  
1        0.296463   0.285809  
21       0.352016   0.275231  
2        0.277331   0.237991  
4        0.279338   0.222507  
11       0.229328   0.220442  
6        0.259008   0.218915  
12       0.235041   0.217702  
16       0.249480   0.157926  
13       0.209161   0.155320  
7        0.214301   0.148128  
3        0.197564   0.136177  
15       0.215537   0.125152  
10       0.199005   0.120085  
14       0.195598   0.119688  
18       0.206312   0.119497  
19       0.182402   0.114390  
24       0.196000   0.100539  
27       0.195356   0.096304  
31       0.163483   0.085651  
28       0.147165   0.080334  
17       0.161616   0.069500  
30       0.139214   0.062530  
8        0.149098   0.059412  
25       0.145582   0.057076  
23       0.148912   0.057007  
26       0.140159   0.053750  
22       0.139367   0.051728  
29       0.136810   0.051656  
20       0.140550   0.051382